In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.python.ops import rnn_cell
from tensorflow.python.ops import seq2seq

In [2]:
# Data
data = open('data/dante.txt', 'r').read()

In [3]:
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
# dict eg. {'a':1} for all chars
vocab = dict(zip(chars, range(len(chars))))
# apply dict.get to all data
# all chars of the text are "encoded" in numbers
tensor = np.array(list(map(vocab.get,data)))

In [4]:
batch_size = 10
seq_len = 10
rnn_size = 10
num_layers = 1

In [5]:
num_batches = int(tensor.size/(batch_size*seq_len))
tensor = tensor[:num_batches*batch_size*seq_len]

In [6]:
xdata = tensor
ydata = np.copy(tensor)
len(ydata)

548200

In [7]:
ydata[:-1] = xdata[1:]
ydata[-1] = xdata[0]

In [8]:
# is a list where each entry is a 10x10 matrix
# each row is an encoded seq of words of size 10
x_batches = np.split(xdata.reshape(batch_size, -1), num_batches,1)
y_batches = np.split(xdata.reshape(batch_size, -1), num_batches,1)

In [9]:
x_data = x_batches[0]
y_data = y_batches[1]

In [10]:
sess = tf.InteractiveSession()

In [11]:
# I/O
x_model = tf.placeholder(tf.int32, [batch_size, seq_len])
y_model = tf.placeholder(tf.int32, [batch_size, seq_len])

In [12]:
# cells
cell = rnn_cell.LSTMCell(rnn_size, state_is_tuple=True, activation= tf.nn.elu)
multi_cell = rnn_cell.MultiRNNCell([cell]*num_layers, state_is_tuple=True)

In [13]:
# init state cell
init = multi_cell.zero_state(batch_size, tf.float32)

In [14]:
# W and bias
softmax_w = tf.get_variable('softmax_w', [rnn_size, vocab_size])
softmax_b = tf.get_variable('softmax_b', [vocab_size])

In [15]:
# word embedding the output is a list of vector of size seq_len
embedding = tf.get_variable('embedding', [vocab_size, rnn_size])
input_lookup = tf.nn.embedding_lookup(embedding, x_model)
inputs = tf.split(1, seq_len, input_lookup)
inputs_list = [tf.squeeze(input_, [1]) for input_ in inputs]

In [16]:
# decoder from tf example
output_seq, last_state = seq2seq.rnn_decoder(inputs_list, init, multi_cell)

In [17]:
output = tf.reshape(tf.concat(1, output_seq), [-1, rnn_size])
logits = tf.matmul(output, softmax_w)+softmax_b

In [18]:
# probs is a prob matrix where each entry is the prob of the correct guess
probs = tf.nn.softmax(logits)

In [19]:
# compute loss across the seq_len
cross_seq = seq2seq.sequence_loss_by_example([logits],
                                         [tf.reshape(y_model,[-1])],
                                        [tf.ones([batch_size*seq_len])],
                                        vocab_size)

In [20]:
# averaging over the seq_len
cost = tf.reduce_sum(cross_seq) / batch_size / seq_len

In [21]:
# model 2 uses dnn 
output_dnn, end_state = tf.nn.dynamic_rnn(multi_cell, input_lookup, initial_state=init)

In [22]:
output_dnn = tf.reshape(output_dnn, [-1, batch_size])
log_dnn = tf.matmul(output_dnn, softmax_w) + softmax_b
prob_dnn = tf.nn.softmax(log_dnn)

In [23]:
y_reshaped = tf.reshape(y_model,[-1])
cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(prob_dnn, y_reshaped)

In [24]:
total_loss = tf.reduce_mean(cross_entropy)
cost = tf.reduce_sum(cross_seq) / batch_size / seq_len

In [25]:
sess.run(tf.global_variables_initializer())
state = sess.run(init)

In [26]:
feed = {x_model:x_data, y_model:y_data}
for i, (c,h) in enumerate(init):
    feed[c] = state[i].c
    feed[h] = state[i].h

In [27]:
cr_dnn, loss_dnn = sess.run([cross_entropy, total_loss], feed)
cr_seq, loss_seq = sess.run([cross_seq, cost], feed)
loss_seq

5.0488653

In [28]:
loss_dnn

4.3963375

In [29]:
sess.close()

In [30]:
def variable_summaries(var):

		with tf.name_scope('summaries'):
			mean = tf.reduce_mean(var)
			tf.summary.scalar('mean',var)

			with tf.name_scope('sd'):
				sd = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
				tf.summary.scalar('sd', sd)
				tf.summary.scalar('max',tf.reduce_max(var))
				tf.summary.scalar('min',tf.reduce_min(var))
				tf.summary.histogram('histogram',var)


In [31]:
variable_summaries(total_loss)

In [32]:
variable_summaries

<function __main__.variable_summaries>